# Entraînement du Modèle U-Net 3D

Ce notebook entraîne un modèle U-Net 3D pour la détection d'anévrismes.

**Étapes** :
1. Chargement du dataset créé
2. Split train/val/test
3. Création des DataLoaders PyTorch
4. Configuration du modèle et de l'entraînement
5. Entraînement avec Trainer
6. Évaluation et visualisation des résultats

In [ ]:
import sys
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

sys.path.append("../")

from src import PROCESSED_DIR, MODELS_DIR
from src.models import UNet3DClassifier
from src.bricks import Trainer

# Seed pour reproductibilité
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

## 1. Chargement du Dataset

Chargement du dataset CTA créé dans le notebook 02.

In [ ]:
import os

# Charger le dataset CTA
dataset_path = os.path.join(PROCESSED_DIR, "cta_dataset.npz")

if os.path.exists(dataset_path):
    loaded = np.load(dataset_path, allow_pickle=True)
    data = {key: loaded[key].item() for key in loaded.files}
    print(f"Dataset chargé: {len(data)} patients")
    print(f"Exemple - patient_0 a {len(data['patient_0']['cubes'])} cubes")
else:
    print(f"Dataset non trouvé : {dataset_path}")
    print("Exécutez d'abord le notebook 02_dataset_creation.ipynb")

## 2. Split Train/Val/Test

Division des données par patient (70% train, 15% val, 15% test).

In [ ]:
# Split par patient
all_patients = list(data.keys())

train_patients, temp_patients = train_test_split(
    all_patients, test_size=0.3, random_state=SEED
)
val_patients, test_patients = train_test_split(
    temp_patients, test_size=0.5, random_state=SEED
)

print(f"{len(train_patients)} train, {len(val_patients)} val, {len(test_patients)} test")

train_data = {k: data[k] for k in train_patients}
val_data = {k: data[k] for k in val_patients}
test_data = {k: data[k] for k in test_patients}

## 3. Dataset PyTorch

Classe Dataset pour charger les cubes en batch.

In [ ]:
class CubesDataset(Dataset):
    """Dataset PyTorch pour cubes 3D."""
    
    def __init__(self, data_dict, transform=None):
        self.transform = transform
        self.cubes = np.concatenate([d['cubes'] for d in data_dict.values()], axis=0)
        self.positions = np.concatenate([d['positions'] for d in data_dict.values()], axis=0)
        self.labels = np.concatenate([d['labels'] for d in data_dict.values()], axis=0)

    def __len__(self):
        return len(self.cubes)

    def __getitem__(self, idx):
        cube = self.cubes[idx]
        label = self.labels[idx]
        position = self.positions[idx]
        
        # Convertir en tenseur PyTorch
        cube = torch.tensor(cube, dtype=torch.float32).unsqueeze(0)
        
        # Concaténer position (13) et label (1) -> shape (14,)
        y = np.concatenate([position, [label]], axis=0)
        y = torch.tensor(y, dtype=torch.float32)
        
        return cube, y

## 4. DataLoaders

Création des DataLoaders pour l'entraînement.

In [ ]:
# Datasets
train_dataset = CubesDataset(train_data)
val_dataset = CubesDataset(val_data)
test_dataset = CubesDataset(test_data)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

# Vérifier les shapes
for cube, y in train_loader:
    print(f"Cube shape: {cube.shape}")
    print(f"Target shape: {y.shape}")
    break

## 5. Loss Function

Loss combinée : BCE pour les positions + BCE pour le label binaire.

In [ ]:
def combined_loss(pred, target, alpha=0.1):
    """Loss combinée pour positions + label.
    
    Parameters
    ----------
    pred : torch.Tensor
        Prédictions (B, 14)
    target : torch.Tensor
        Targets (B, 14)
    alpha : float
        Poids pour la loss de position
    
    Returns
    -------
    torch.Tensor
        Loss combinée
    """
    pos_pred = torch.sigmoid(pred[:, :13])
    pos_target = target[:, :13]
    label_pred = pred[:, 13:]
    label_target = target[:, 13:]
    
    loss_pos = F.binary_cross_entropy(pos_pred, pos_target)
    loss_label = F.binary_cross_entropy_with_logits(label_pred, label_target)
    
    return alpha * loss_pos + loss_label

## 6. Configuration du Modèle

Utilisation de UNet3DClassifier depuis src.models.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Modèle
model = UNet3DClassifier(in_ch=1, base_ch=32)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print(f"Modèle: {model}")
print(f"Nombre de paramètres: {sum(p.numel() for p in model.parameters()):,}")

## 7. Entraînement avec Trainer

Utilisation de la classe Trainer depuis src.bricks pour l'entraînement.

In [ ]:
# Créer le trainer
trainer = Trainer(
    model=model,
    criterion=combined_loss,
    optimizer=optimizer,
    device=device,
    checkpoint_dir=MODELS_DIR
)

# Entraîner
trainer.fit(train_loader, val_loader, epochs=5)

## 8. Visualisation des Résultats

Affichage des courbes de loss et accuracy.

In [ ]:
# Visualiser l'historique
trainer.plot_history()

## 9. Sauvegarde du Modèle

Sauvegarde du modèle entraîné.

In [ ]:
# Sauvegarder le modèle final
import os
final_model_path = os.path.join(MODELS_DIR, "unet3d_cta_final.pth")
torch.save(model.state_dict(), final_model_path)
print(f"Modèle final sauvegardé : {final_model_path}")

## 10. Évaluation sur Test Set

Évaluation finale sur le test set.

In [ ]:
# Évaluer sur test set
test_loss, test_acc = trainer.validate(test_loader)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")